# 1.4 SageMaker Training with MLFlow, HPO and Processing

## 학습 작업의 실행 노트북 개요

- SageMaker Training에 SageMaker 실험을 추가하여 여러 실험의 결과를 비교할 수 있습니다.
    - [작업 실행 시 필요 라이브러리 import](#작업-실행-시-필요-라이브러리-import)
    - [SageMaker 세션과 Role, 사용 버킷 정의](#SageMaker-세션과-Role,-사용-버킷-정의)
    - [하이퍼파라미터 정의](#하이퍼파라미터-정의)
    - [학습 실행 작업 정의](#학습-실행-작업-정의)
        - 학습 코드 명
        - 학습 코드 폴더 명
        - 학습 코드가 사용한 Framework 종류, 버전 등
        - 학습 인스턴스 타입과 개수
        - SageMaker 세션
        - 학습 작업 하이퍼파라미터 정의
        - 학습 작업 산출물 관련 S3 버킷 설정 등
    - [학습 데이터셋 지정](#학습-데이터셋-지정)
        - 학습에 사용하는 데이터셋의 S3 URI 지정
    - [SageMaker MLFlow 설정](#SageMaker-MLFlow-설정)
    - [학습 실행](#학습-실행)
    - [데이터 세트 설명](#데이터-세트-설명)
    - [실험 결과 보기](#실험-결과-보기)

In [ ]:
%store -r
print(f"tracking_server_name : {tracking_server_name}")

### 작업 실행 시 필요 라이브러리 import

In [ ]:
import boto3
import sagemaker

### SageMaker 세션과 Role, 사용 버킷 정의

In [ ]:
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

In [ ]:
bucket = sagemaker_session.default_bucket()
code_location = f's3://{bucket}/xgboost/code'
output_path = f's3://{bucket}/xgboost/output'

### 하이퍼파라미터 정의

In [ ]:
hyperparameters = {
       "scale_pos_weight" : "29",    
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}

### 학습 실행 작업 정의

In [ ]:
instance_count = 1
instance_type = "ml.m5.large"
# instance_type = "local"
max_run = 1*60*60

use_spot_instances = False
if use_spot_instances:
    max_wait = 1*60*60
else:
    max_wait = None

In [ ]:
if instance_type in ['local', 'local_gpu']:
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
else:
    sagemaker_session = sagemaker.session.Session()

In [ ]:
from sagemaker.xgboost.estimator import XGBoost

estimator = XGBoost(
    entry_point="xgboost_script_mlflow.py",
    source_dir="src",
    output_path=output_path,
    code_location=code_location,
    hyperparameters=hyperparameters,
    role=role,
    sagemaker_session=sagemaker_session,
    instance_count=instance_count,
    instance_type=instance_type,
    framework_version="1.7-1",
    max_run=max_run,
    use_spot_instances=use_spot_instances,  # spot instance 활용
    max_wait=max_wait,
)

### 학습 데이터셋 지정

In [ ]:
data_path=f's3://{bucket}/xgboost/dataset'
!aws s3 sync ../data/dataset/ $data_path

In [ ]:
if instance_type in ['local', 'local_gpu']:
    from pathlib import Path
    file_path = f'file://{Path.cwd()}'
    inputs = file_path.split('lab_1_training')[0] + 'data/dataset/'
    
else:
    inputs = data_path
inputs

### SageMaker MLFlow 설정

In [ ]:
import mlflow
import mlflow.xgboost
import os
from time import strftime

sm_client = boto3.client('sagemaker')

In [ ]:
# 추적 서버 설명 가져오기
tracking_server = sm_client.describe_mlflow_tracking_server(
    TrackingServerName=tracking_server_name
)

# 추적 서버 URI 가져오기
tracking_server_uri = tracking_server['TrackingServerArn']

# MLflow 추적 서버 URI 설정
mlflow.set_tracking_uri(tracking_server_uri)

In [ ]:
experiment_name = 'xgboost-poc-1'

# MLflow 실험 생성 또는 가져오기
mlflow.set_experiment(experiment_name)


### HPO 실행 + 학습 실행

SageMaker의 [Automatic Model Tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html)을 활용할 수 있습니다. 이 방식은 높은 평가 비용의 최적화 문제를 위해 특별히 설계된 베이지안 최적화 방법을 사용합니다. [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/tuner.html)의 `fit()` 방법은 `Estimator`와 같이 기본적으로 제공되지 않습니다. (HPO 작업은 일반적으로는 오래 걸리기 때문입니다.) SageMaker console에 있는 "Hyperparameter Tuning Jobs"은 진행되는 작업의 상세 상태와 metrics를 확인하기에 좋은 UI를 제공합니다. 

In [ ]:
max_jobs=4    # TODO: Ideally 12 or more
max_parallel_jobs=2   # TODO: Maybe only 1 for Event Engine, 2-3 if possible

### 학습 실행

In [ ]:

# 현재 실행 시간을 기반으로 고유한 실행 이름 생성
create_date = strftime("%H%M%s")
run_name = f'{experiment_name}-{create_date}'

# MLflow 실행 시작
with mlflow.start_run(run_name=run_name) as run:
    run_id = run.info.run_id
    
    # 하이퍼파라미터 로깅
    for key, value in hyperparameters.items():
        mlflow.log_param(key, value)
    
    # 인스턴스 정보 로깅
    mlflow.log_param("instance_type", instance_type)
    mlflow.log_param("instance_count", instance_count)
    
    # 입력 데이터 경로 로깅
    mlflow.log_param("input_data_path", inputs)
    
    # SageMaker 훈련 작업에 MLflow 환경 변수 전달
    estimator.environment = {
            "MLFLOW_TRACKING_URI": tracking_server_uri,
            "MLFLOW_EXPERIMENT_NAME": experiment_name,
            "MLFLOW_RUN_ID": run_id
    }

    tuner = sagemaker.tuner.HyperparameterTuner(
        estimator,
        objective_metric_name="validation:auc",
        hyperparameter_ranges={
            "max_depth": sagemaker.tuner.IntegerParameter(2, 5),
            "eta": sagemaker.tuner.ContinuousParameter(0.1, 0.5)
        },
        objective_type="Maximize",
        max_jobs=max_jobs,    # TODO: Ideally 12 or more
        max_parallel_jobs=max_parallel_jobs,    # TODO: Maybe only 1 for Event Engine, 2-3 if possible
    )
    
    tuner.fit(
        job_name = run_name,
        inputs={'inputdata': inputs},
        wait=False
    )
    
    # MLflow에 모델 아티팩트 경로 로깅
    mlflow.log_param("model_artifact_path", f"{output_path}/{run_name}/output/model.tar.gz")

In [ ]:
tuner.wait()

###  실험 결과 보기
위의 실험한 결과를 확인 합니다.
- 각각의 훈련잡의 시도에 대한 훈련 사용 데이터, 모델 입력 하이퍼 파라미터, 모델 평가 지표, 모델 아티펙트 결과 위치 등의 확인이 가능합니다.
- **아래의 모든 내용은 SageMaker Studio 를 통해서 직관적으로 확인이 가능합니다.**

In [ ]:
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 10
pd.options.display.max_colwidth = 100

In [ ]:
# MLflow 실험 결과 가져오기
from mlflow.tracking import MlflowClient

client = MlflowClient()

# 실험 결과 조회
try:
    experiment = client.get_experiment_by_name(experiment_name)
    if experiment:
        experiment_id = experiment.experiment_id
        
        # 실행 결과 검색 (정렬 없이)
        runs = client.search_runs(experiment_ids=[experiment_id])
        
        if runs:
            print(f"실험 '{experiment_name}'에서 {len(runs)}개의 실행 결과를 찾았습니다.")
            
            # 사용 가능한 메트릭 이름 확인
            print("사용 가능한 메트릭 이름:")
            for key in runs[0].data.metrics.keys():
                print(f"- {key}")
            
            # 결과를 데이터프레임으로 변환
            runs_data = []
            for run in runs:
                run_data = {
                    "run_id": run.info.run_id,
                    "run_name": run.info.run_name,
                    "status": run.info.status,
                    "start_time": run.info.start_time,
                    "end_time": run.info.end_time,
                }
                
                # 메트릭 추가
                for key, value in run.data.metrics.items():
                    run_data[key] = value
                
                # 파라미터 추가
                for key, value in run.data.params.items():
                    run_data[f"param_{key}"] = value
                
                runs_data.append(run_data)
            
            runs_df = pd.DataFrame(runs_data)
            
            # 파이썬에서 정렬 (validation_auc 열이 있는 경우)
            if 'validation_auc' in runs_df.columns:
                runs_df = runs_df.sort_values(by='validation_auc', ascending=False)
                
            display(runs_df)
        else:
            print("실험에 실행 결과가 없습니다.")
    else:
        print(f"실험 '{experiment_name}'을 찾을 수 없습니다.")
except Exception as e:
    print(f"실험 결과 조회 중 오류 발생: {e}")

###  Evaluation 하기
SageMaker Processing을 이용하여 Evalution을 수행하는 코드를 동작할 수 있습니다. MLOps에서 Processing을 적용하면 전처리, Evaluation 등을 serverless로 동작할 수 있습니다.

In [ ]:
from sagemaker.processing import FrameworkProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from datetime import datetime

In [ ]:
instance_count = 1
instance_type = "ml.m5.large"
# instance_type = 'local'

In [ ]:
script_eval = FrameworkProcessor(
    XGBoost,
    framework_version="1.7-1",
    role=role,
    instance_type=instance_type,
    instance_count=instance_count
)

In [ ]:
response = sm_client.describe_training_job(
    TrainingJobName=tuner.best_training_job()
)

In [ ]:
artifacts_dir = response['ModelArtifacts']['S3ModelArtifacts']
artifacts_dir

In [ ]:
s3_test_path = data_path + '/test.csv'
detect_outputpath = f's3://{bucket}/xgboost/processing'

In [ ]:
from pathlib import Path
source_dir = f'{Path.cwd()}/src'

if instance_type == 'local':
    from sagemaker.local import LocalSession
    from pathlib import Path

    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}

    s3_test_path=f'../data/dataset/test.csv'
else:
    sagemaker_session = sagemaker.session.Session()
    s3_test_path=data_path + '/test.csv'  

In [ ]:
# MLflow 실험 생성 및 설정
def create_experiment(experiment_name):
    try:
        mlflow.create_experiment(experiment_name)
    except:
        pass  # 이미 존재하는 경우 무시
    return experiment_name

# SageMaker 실험 생성 함수
def create_trial(experiment_name):
    # 현재 시간을 기반으로 고유한 실행 이름 생성
    run_name = f"processing-job-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"
    
    # SageMaker 실험이 존재하는지 확인하고 없으면 생성
    try:
        sm_client.describe_experiment(ExperimentName=experiment_name)
    except sm_client.exceptions.ResourceNotFound:
        sm_client.create_experiment(ExperimentName=experiment_name)
    
    # SageMaker Trial 생성
    try:
        sm_client.create_trial(
            TrialName=run_name,
            ExperimentName=experiment_name,
            DisplayName=run_name
        )
    except Exception as e:
        print(f"Error creating trial: {e}")
        # 이미 존재하는 경우 무시하거나 다른 이름으로 시도
        run_name = f"processing-job-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}-{int(time.time())}"
        sm_client.create_trial(
            TrialName=run_name,
            ExperimentName=experiment_name,
            DisplayName=run_name
        )
    
    return run_name

In [ ]:
# 메인 실행 시작
experiment_name = create_experiment(experiment_name)
run_name = create_trial(experiment_name)  # SageMaker Trial 생성

# MLflow 설정
mlflow.set_tracking_uri(tracking_server_uri)
mlflow.set_experiment(experiment_name)

# MLflow 실행 시작
with mlflow.start_run(run_name=run_name) as run:
    run_id = run.info.run_id
    
    # 처리 작업 실행
    script_eval.run(
        code="evaluation_mlflow.py",
        source_dir=source_dir,
        inputs=[
            ProcessingInput(source=s3_test_path, input_name="test_data", destination="/opt/ml/processing/test"),
            ProcessingInput(source=artifacts_dir, input_name="model_weight", destination="/opt/ml/processing/model")
        ],
        outputs=[
            ProcessingOutput(source="/opt/ml/processing/output", output_name='evaluation', destination=detect_outputpath + "/" + run_name),
        ],
        arguments=[
            "--tracking-uri", tracking_server_uri,
            "--experiment-name", experiment_name,
            "--run-id", run_id
        ],
        job_name=run_name,
        experiment_config={
            'ExperimentName': experiment_name,
            'TrialName': run_name,
            'TrialComponentDisplayName': run_name,
        },
        wait=False
    )

In [ ]:
script_eval.latest_job.wait()

###  실험 결과 확인

In [ ]:
import subprocess
import os

def extract_tar_file(tar_path, extract_path):
    """경고 메시지를 숨기고 tar 파일 추출"""
    os.makedirs(extract_path, exist_ok=True)
    try:
        # stderr를 /dev/null로 리디렉션하여 경고 숨기기
        subprocess.check_call(
            f"tar -xf {tar_path} -C {extract_path} 2>/dev/null", 
            shell=True
        )
        return True
    except subprocess.CalledProcessError:
        print(f"tar 파일 {tar_path} 추출 중 오류 발생")
        return False

In [ ]:
!aws s3 ls --human-readable {artifacts_dir}/
artifacts_dir

In [ ]:
model_dir = './model'

!rm -rf $model_dir

import json , os

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

!aws s3 cp {artifacts_dir} {model_dir}/model.tar.gz
!tar --warning=no-unknown-keyword -xvzf {model_dir}/model.tar.gz -C {model_dir}

In [ ]:
!pip install xgboost graphviz

In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt

In [ ]:
model = xgb.XGBClassifier()
model.load_model("./model/xgboost-model")

In [ ]:
test_prep_df = pd.read_csv('../data/dataset/test.csv')
x_test = test_prep_df.drop('fraud', axis=1)
feature_data = xgb.DMatrix(x_test)
model.get_booster().feature_names = feature_data.feature_names
model.get_booster().feature_types = feature_data.feature_types
fig, ax = plt.subplots(figsize=(15, 8))
xgb.plot_importance(model, ax=ax, importance_type='gain')

In [ ]:
xgb.plot_tree(model, num_trees=0, rankdir='LR')

fig = plt.gcf()
fig.set_size_inches(50, 15)
plt.show()